In [1]:
# !pip install -q evaluate
# !pip install -q rouge_score

## Import Libraries

In [2]:
import nltk
import evaluate
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset,DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,T5Tokenizer,T5ForConditionalGeneration
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer,  DataCollatorForTokenClassification
from tqdm import tqdm
import evaluate
import os
os.environ["WANDB_DISABLED"] = "true"

## Load flan-t5 model

In [3]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "/root/autodl-tmp/models/codet5-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Split dataset for training

In [4]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv', index_col=0, encoding='utf_8_sig')
# df.fillna('', inplace=True)
# label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
# df = df.rename(columns={'labels':'label','msgs':'message','diffs':'diff'})
# df = df.replace({"labels": label2id})
# df = df['diffs']
# # print(df)
df['text'] = df['msgs'] + df['diffs']
df
# test_sample = df.sample(n=3, random_state=1)

,repo,commit,labels,msgs,diffs,feature,text
user,,,,,,,
ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,Perfective,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Change hasException to hasThrowable--diff --gi...
ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,Adaptive,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ...",Trying to extend the Scheduler interface accor...
ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,Adaptive,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ...",RunAsync method for outputting multiple values...
ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,Corrective,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,...",forEach with Action1 but not Observer--I re-re...
ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,Corrective,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,...",1.x: Fix Completable swallows- OnErrorNotImple...
...,...,...,...,...,...,...,...
jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,Perfective,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",Use a temporary file to decrease memory consum...
jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,Perfective,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,...",added a more memory efficient structure for st...
hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,Perfective,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17...",Memory and performance optimizationsdiff --git...


In [5]:
full_dataset = Dataset.from_pandas(df)

# 1) train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
first_split = full_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = first_split["train"]      # ~70%
test_tmp  = first_split["test"]       # ~30%

# 2) test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
second_split = test_tmp.train_test_split(test_size=0.5, seed=42)
test_dataset  = second_split["train"]     # ~15%
valid_dataset   = second_split["test"]      # ~15%  （后面会被覆盖，和你 pandas 逻辑一致）

ds_splits = DatasetDict({
    "train": train_dataset,
    "valid": valid_dataset,
    "test":  test_dataset,
})

In [6]:
# train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
# test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
# train_data,_ = train_test_split(train_data, train_size=0.1, random_state=42)
# test_data, val_data = train_test_split(test_data, train_size=0.1, random_state=42)
# val_data, _ = train_test_split(val_data, train_size=0.1, random_state=42)

In [7]:
print("Before:\n", full_dataset)
print("After\n", ds_splits)

Before:
 Dataset({
    features: ['repo', 'commit', 'labels', 'msgs', 'diffs', 'feature', 'text', 'user'],
    num_rows: 1781
})
After
 DatasetDict({
    train: Dataset({
        features: ['repo', 'commit', 'labels', 'msgs', 'diffs', 'feature', 'text', 'user'],
        num_rows: 1246
    })
    valid: Dataset({
        features: ['repo', 'commit', 'labels', 'msgs', 'diffs', 'feature', 'text', 'user'],
        num_rows: 268
    })
    test: Dataset({
        features: ['repo', 'commit', 'labels', 'msgs', 'diffs', 'feature', 'text', 'user'],
        num_rows: 267
    })
})


## Post-processing

In [8]:
# int_to_str = {0:'0', 1:'1'}

# prefix = "Please check whether the tweet is about a real disaster or not: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = examples["text"]
   model_inputs = tokenizer(inputs, max_length=128, padding=True, truncation=True)
    
   labels = tokenizer(text_target=examples["labels"], padding=True, truncation=True)
   model_inputs["labels"] = labels["input_ids"]

   return model_inputs

In [9]:
# Map the preprocessing function across our dataset
tokenized_dataset = ds_splits.map(preprocess_function, batched=False, remove_columns=['text','labels'])

Map:   0%|          | 0/1246 [00:00<?, ? examples/s]

Map:   0%|          | 0/268 [00:00<?, ? examples/s]

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

## Evaluation using ROGUE Score

In [10]:
# nltk.download("punkt", quiet=True)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

str_to_int = {'Adaptive':0, 'Corrective':1, 'Perfective':2}

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # 替换 label 中的 -100 为 pad，避免 decode 出错
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # 解码成字符串
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 去掉首尾空格
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    # string → int (positive / negative)
    y_pred = [str_to_int.get(p, -1) for p in decoded_preds]
    y_true = [str_to_int.get(l, -1) for l in decoded_labels]

    # 计算四个指标
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


## Training

In [11]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 32
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 10

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   eval_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   logging_steps = 100
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

/tmp/ipykernel_3050/2420821063.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.314510,0.415730,0.367430,0.423004,0.316986
2,No log,0.252205,0.419476,0.276980,0.419978,0.333806
3,0.510600,0.228487,0.516854,0.653766,0.497441,0.477721
4,0.510600,0.239595,0.430712,0.395936,0.440781,0.340719
5,0.510600,0.248345,0.539326,0.634936,0.568694,0.480303
6,0.233800,0.210336,0.677903,0.688826,0.690530,0.677242
7,0.233800,0.161429,0.764045,0.767265,0.762306,0.763988


/root/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/root/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [17]:
fewshot_metrics = trainer.evaluate(eval_dataset=tokenized_dataset["valid"])
fewshot_metrics

{'eval_loss': 0.16809788346290588,
 'eval_accuracy': 0.8134328358208955,
 'eval_precision': 0.8145611937155449,
 'eval_recall': 0.8143576156211338,
 'eval_f1': 0.8134502923976609,
 'eval_runtime': 2.3249,
 'eval_samples_per_second': 115.275,
 'eval_steps_per_second': 14.624,
 'epoch': 10.0}